In [29]:
import babi_input
import numpy as np

In [32]:
class Config(object):
    """Holds model hyperparams and data information."""

    batch_size = 100
    embed_size = 81
    hidden_size = 83

    max_epochs = 256
    early_stopping = 20

    dropout = 0.9
    lr = 0.001
    l2 = 0.001

    cap_grads = False
    max_grad_val = 10
    noisy_grads = False

    word2vec_init = False
    embedding_init = np.sqrt(3)

    # set to zero with strong supervision to only train gates
    strong_supervision = False
    beta = 1

    drop_grus = False

    anneal_threshold = 1000
    anneal_by = 1.5

    num_hops = 3
    num_attention_features = 4

    max_allowed_inputs = 130
    num_train = 900

    floatX = np.float32

    babi_id = "1"
    babi_test_id = ""

    train_mode = True


In [27]:
def load_data(config):
    """Loads train/valid/test data and sentence encoding"""
    if config.train_mode:
        (train, valid, word_embedding,
         max_q_len, max_input_len, max_sen_len, 
         num_supporting_facts, vocab_size) = babi_input.load_babi(config, split_sentences=True)
        return train
    else:
        (test, word_embedding,
         max_q_len, max_input_len, max_sen_len,
         num_supporting_facts, vocab_size) = babi_input.load_babi(config, split_sentences=True)
        return test

In [36]:
config = Config()
data_train = load_data(config)

config.train_mode = False
data_test = load_data(config)

NameError: name 'load_data' is not defined

In [38]:
data_train[1].shape

(1000, 10, 6)

In [32]:
data_train[0].shape

(1000, 3)

In [35]:
data_test[0].shape

(1000, 3)

In [2]:
import tensorflow as tf

import time
import argparse
import os

parser = argparse.ArgumentParser()
parser.add_argument("-f", "--files", help="specify babi task 1-20 (default=1)")
parser.add_argument("-b", "--babi_task_id", help="specify babi task 1-20 (default=1)")
parser.add_argument("-r", "--restore", help="restore previously trained weights (default=false)")
parser.add_argument("-s", "--strong_supervision", help="use labelled supporting facts (default=false)")
parser.add_argument("-t", "--dmn_type", help="specify type of dmn (default=original)")
parser.add_argument("-l", "--l2_loss", type=float, default=0.001, help="specify l2 loss constant")
parser.add_argument("-n", "--num_runs", type=int, help="specify the number of model runs")

args = parser.parse_args()


In [13]:
tf.reset_default_graph()

In [14]:

dmn_type = args.dmn_type if args.dmn_type is not None else "plus"

if dmn_type == "original":
    from dmn_original import Config
    config = Config()
elif dmn_type == "plus":
    from dmn_plus import Config
    config = Config()
else:
    raise NotImplementedError(dmn_type + ' DMN type is not currently implemented')

if args.babi_task_id is not None:
    config.babi_id = args.babi_task_id

config.babi_id = args.babi_task_id if args.babi_task_id is not None else str(1)
config.l2 = args.l2_loss if args.l2_loss is not None else 0.001
config.strong_supervision = args.strong_supervision if args.strong_supervision is not None else False
num_runs = args.num_runs if args.num_runs is not None else 1

print('Training DMN %s on babi task %s' % (dmn_type, config.babi_id))

best_overall_val_loss = float('inf')

# create model
with tf.variable_scope('DMN') as scope:
    if dmn_type == "original":
        from dmn_original import DMN
        model = DMN(config)
    elif dmn_type == "plus":
        print('loading plus')
        from dmn_plus import DMN_PLUS
        model = DMN_PLUS(config)


Training DMN plus on babi task 1
loading plus
==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt
==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt
==> get train inputs
==> get test inputs
size of training data: (1000, 3)
==> get question representation
==> get input representation
==> build episodic memory
==> generating episode 0
==> generating episode 1
==> generating episode 2


In [7]:
len(model.train[0])

1000

In [37]:
config.num_train

900

In [17]:
model.config.babi_id

'1'

In [38]:
(train, valid, word_embedding,
 max_q_len, max_input_len, max_sen_len,
 num_supporting_facts, vocab_size) = babi_input.load_babi(
    config, split_sentences=True)

==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt
==> Loading test from /home/dong/Dropbox/Projects/NLP/memoryNet/bAbIdata/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt
==> get train inputs
==> get test inputs


In [40]:
valid[0].shape

(100, 3)